# Haithm Style Fine-Tuning (Qwen 3B QLoRA)

This notebook fine-tunes the Qwen 2.5 3B Instruct model on Haithm's personal writing style using QLoRA.

## 1. Setup
**Before running:**
1. Upload your dataset files (`dataset_haithm_style_natural.jsonl`, `dataset_haithm_style_prompts.jsonl`) to the Colab runtime (drag & drop to file sidebar).
2. Enable GPU Runtime: `Runtime` -> `Change runtime type` -> `T4 GPU` (or better).

In [ ]:
!pip install -q -U torch transformers peft datasets bitsandbytes trl

## 2. Load Model & Dataset

In [ ]:
import torch
from datasets import load_dataset, concatenate_datasets
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer

# Config
model_name = "Qwen/Qwen2.5-3B-Instruct"
dataset_natural = "dataset_haithm_style_natural.jsonl"
dataset_prompts = "dataset_haithm_style_prompts.jsonl"
new_model = "qwen-3b-haithm-style-lora"

# Load Datasets
print("Loading datasets...")
dataset = load_dataset("json", data_files={"train": dataset_natural}, split="train")

try:
    prompts_ds = load_dataset("json", data_files={"train": dataset_prompts}, split="train")
    # Oversample prompts (x5) to ensure they have impact
    for _ in range(5):
        dataset = concatenate_datasets([dataset, prompts_ds])
    print(f"Merged datasets. Total examples: {len(dataset)}")
except Exception as e:
    print(f"Warning: Could not load prompts dataset: {e}")

# 4-bit Quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

# Load Base Model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

## 3. Training (QLoRA)

In [ ]:
# LoRA Config
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)

# Training Params
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
)

# Traverse
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="output",
    max_seq_length=2048,
    tokenizer=tokenizer,
    args=training_args,
    packing=False,
)

trainer.train()

## 4. Save Adapter

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

trainer.model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)

print(f"Model saved to {new_model}")

# Zip for download
!zip -r {new_model}.zip {new_model}
print("Download the zip file from the file browser.")